**Imports**

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

import torch
from torch import nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

**Classes do Modelo ANN**

In [2]:
# Solar ANN

class SolarANN(nn.Module):
    def __init__(self, input_dim, neurons, dropouts, activation_function_per_layer):
        """
        Constrói a rede neural dinamicamente baseada nas listas de neurônios e dropouts.
        """
        super(SolarANN, self).__init__()
        
        layers = []
        in_dim = input_dim
        function = {
            'relu'   : nn.ReLU(),
            'sigmoid': nn.Sigmoid(),
            'tanh'   : nn.Tanh()
        }

        # Cria as camadas ocultas dinamicamente
        for out_dim, drop_rate, activation_function in zip(neurons, dropouts, activation_function_per_layer):
            layers.append(nn.Linear(in_dim, out_dim))
            layers.append(function[activation_function])
            if drop_rate > 0:
                layers.append(nn.Dropout(drop_rate))
            in_dim = out_dim
            
        layers.append(nn.Linear(in_dim, 1))
        
        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)
    
class SolarPrediction:
    def __init__(self, nome_cidade, config):
        self.nome = nome_cidade
        self.config = config
        self.df = None
        self.scalers = {} # Para guardar o scaler de cada target se necessário
        self.loaders = {}
        self.input_dim = 0
        
        # Carrega os dados imediatamente ao instanciar
        self._carregar_dados()

    def _carregar_dados(self):
        path = self.config["arquivo"]
        tipo = self.config["tipo"]
        
        # Lógica encapsulada das funções anteriores
        if tipo == "wb":
            df = pd.read_csv(path, sep=';')
            df['time'] = pd.to_datetime(df['time'])

            df['year'] = df['time'].dt.year
            df['month'] = df['time'].dt.month
            df['day'] = df['time'].dt.day
            df['hour'] = df['time'].dt.hour
            df['minute'] = df['time'].dt.minute

            df.drop(columns=['time', 'comments'], inplace=True)

            return df[
                ['year','month','day','hour','minute','air_temperature',
                'relative_humidity','wind_speed','wind_from_direction',
                'wind_speed_calc','sensor_cleaning','precipitation',
                'barometric_pressure','dhi_rsi','ghi_sil','ghi_pyr']
            ]

        elif tipo == "tmy":
            df = pd.read_csv(
                path, sep=',', skiprows=8, skipfooter=12, engine='python'
            )

            df['time'] = pd.to_datetime(df['time'], format="%Y%m%d:%H%M")

            df['year'] = df['time'].dt.year
            df['month'] = df['time'].dt.month
            df['day'] = df['time'].dt.day
            df['hour'] = df['time'].dt.hour

            df.drop(columns=['time','Int','Gr(i)'], inplace=True)

            df = df.rename(columns={ 'Gb(i)': 'GSR', 'Gd(i)': 'DSR' })

            return df[['year','month','day','hour','H_sun','T2m','WS10m','GSR','DSR']]
            
        elif tipo == "sarah":
            df = pd.read_csv(path, sep=';')

            df['time'] = pd.to_datetime(df['time'], format="%d/%m/%Y")

            df['year'] = df['time'].dt.year
            df['month'] = df['time'].dt.month
            df['day'] = df['time'].dt.day

            df.drop(columns=['time'], inplace=True)

            return df[['year','month','day','SDU','DNI']]
            
        self.df = self.df.dropna()

    def preparar_dataloaders(self, target_col, batch_size):
        """
        Prepara X e y, normaliza e cria os DataLoaders para um target específico.
        """
        ignorar = self.config["ignorar"]
        
        # Separa features e target
        X = self.df.drop(columns=ignorar).values
        y = self.df[target_col].values.reshape(-1, 1)

        # Normalização
        scaler_X = StandardScaler()
        scaler_y = StandardScaler()
        
        X = scaler_X.fit_transform(X)
        y = scaler_y.fit_transform(y)
        
        # Guarda o scaler para desnormalizar depois
        self.scalers[target_col] = scaler_y
        self.input_dim = X.shape[1]

        # Split treino/teste (séries temporais não deve ter shuffle no split temporal)
        n = len(self.df)
        t = int(self.config['training'] * n)
        
        X_train = torch.tensor(X[:t], dtype=torch.float32)
        y_train = torch.tensor(y[:t], dtype=torch.float32)
        X_test = torch.tensor(X[t:], dtype=torch.float32)
        y_test = torch.tensor(y[t:], dtype=torch.float32)

        train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=batch_size, shuffle=False)
        test_loader = DataLoader(TensorDataset(X_test, y_test), batch_size=batch_size, shuffle=False)
        
        return train_loader, test_loader
    
    def treinar_modelo(self, model, train_loader, epochs, lr, optimizer_name="adam"):
        """
        Executa o loop de treinamento para um modelo e loader específicos.
        """
        criterion = nn.MSELoss()
        
        if optimizer_name == "adam":
            optimizer = torch.optim.Adam(model.parameters(), lr=lr)
        else:
            optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9)

        best_loss = np.inf
        best_state = None
        
        print(f"Iniciando treino para {self.nome}...")

        for epoch in range(epochs):
            model.train()
            epoch_loss = 0
            for X_batch, y_batch in train_loader:
                optimizer.zero_grad()
                outputs = model(X_batch)
                loss = criterion(outputs, y_batch)
                loss.backward()
                optimizer.step()
                epoch_loss += loss.item()
            
            epoch_loss /= len(train_loader)
            
            if epoch_loss < best_loss:
                best_loss = epoch_loss
                best_state = model.state_dict()
                
            print(f"Epoch {epoch+1}/{epochs} Loss: {epoch_loss:.4f}")
            
        return best_state, best_loss

**Main**

In [ ]:
def main()
    CIDADES = {
    "Touba": {
        "arquivo": "solar-measurementssenegal-toubaifcqc.csv",
        "tipo": "wb",
        "targets": ["dhi_rsi", "ghi_pyr", "ghi_sil"],
        "ignorar": ["dhi_rsi", "ghi_pyr", "ghi_sil"],
        "neurons": [[50, 50], [100, 200], [100, 50]],
        "dropouts": [[0.25, 0.25], [0.25, 0.25], [0, 0]],
        "epochs": [20, 40, 30],
        "batch_size": [128, 128, 128]
    },

    "Fatick": {
        "arquivo": "solar-measurementssenegal-fatickifcqc.csv",
        "tipo": "wb",
        "targets": ["dhi_rsi", "ghi_pyr", "ghi_sil"],
        "ignorar": ["dhi_rsi", "ghi_pyr", "ghi_sil"],
        "neurons": [[100, 50], [100, 50], [50, 50]],
        "dropouts": [[0.25, 0.25], [0.0, 0.25], [0.25, 0.25]],
        "epochs": [20, 50, 150],
        "batch_size": [128, 128, 128]
    },

    "SA Northern Cape": {
        "arquivo": "Timeseries_SA_northern_cape_2005_2016.csv",
        "tipo": "tmy",
        "targets": ["GSR"],
        "ignorar": ["GSR", "DSR"],
        "neurons": [[100, 50]],
        "dropouts": [[0, 0]],
        "epochs": [20],
        "batch_size": [512]
    },

    "CAR Vakaga": {
        "arquivo": "Timeseries_CAR_vakaga_2005_2016.csv",
        "tipo": "tmy",
        "targets": ["GSR"],
        "ignorar": ["GSR", "DSR"],
        "neurons": [[200, 200, 100]],
        "dropouts": [[0, 0, 0]],
        "epochs": [30],
        "batch_size": [512]
    },

    "Egypt Mut": {
        "arquivo": "Timeseries_egypt_mut_2005_2016.csv",
        "tipo": "tmy",
        "targets": ["GSR"],
        "ignorar": ["GSR", "DSR"],
        "neurons": [[200, 200, 100]],
        "dropouts": [[0, 0, 0]],
        "epochs": [7],
        "batch_size": [128]
    },

    "Algeria Tamanrasset": {
        "arquivo": "Timeseries_tamaransset_2005_2016.csv",
        "tipo": "tmy",
        "targets": ["GSR"],
        "ignorar": ["GSR", "DSR"],
        "neurons": [[100, 100, 50]],
        "dropouts": [[0, 0, 0]],
        "epochs": [100],
        "batch_size": [512]
    },

    "Nigeria Borno": {
        "arquivo": "Timeseries_nigeria_borno_2005_2016.csv",
        "tipo": "tmy",
        "targets": ["DSR"],
        "ignorar": ["GSR", "DSR"],
        "neurons": [[100, 50]],
        "dropouts": [[0, 0]],
        "epochs": [50],
        "batch_size": [512]
    },

    "Nigeria Abuja": {
        "arquivo": "SARAH_nigeria_abuja.csv",
        "tipo": "sarah",
        "targets": ["DNI"],
        "ignorar": ["DNI"],
        "neurons": [[200, 200, 50]],
        "dropouts": [[0, 0, 0]],
        "epochs": [100],
        "batch_size": [128]
    },

    "Nigeria Akure": {
        "arquivo": "SARAH_nigeria_akure.csv",
        "tipo": "sarah",
        "targets": ["DNI"],
        "ignorar": ["DNI"],
        "neurons": [[200, 200, 100]],
        "dropouts": [[0, 0, 0]],
        "epochs": [100],
        "batch_size": [128]
    }
}

OTIMIZADORES = ["adam", "sgd"]

LRS = [1e-3, 5e-4, 1e-4]